# Docking with AutoDock Vina

Perform docking with the AutoDock Vina Python API using the "vina" scoring function.


### Libraries for the IQB workshop

| Library         | abbreviation | Purpose |
|:-------------|:---------:|:------------|
| os           | N/A      | operating system functions - handling file paths and directories. |
| MDAnalysis     | mda | molecular dynamics library - used for reading/writing files and selecting atoms |
| vina | vina | AutoDock Vina software for Python and Jupyter notebooks |
| prolif | plf | ProLIF (Protein-Ligand Interaction Fingerprints) - generates interaction fingerprints for complexes made of ligands, protein, DNA or RNA molecules


## Preparing for Docking: Defining a Ligand Box

First, define the binding pocket.
This is where the software will attempt to bind the ligand.
In some cases, people might use software for finding pockets in proteins to define binding sites.
But, the structure retrieved from the PDB already had a ligand bound.
To define the binding box, take the position of the bound ligand from the original structure.

MDAnalysis has tools that can measure molecule and define a binding box.
The approach is to find the `center_of_geometry` of the ligand to define the center of the binding pocket.
Then consider the space around the ligand to be the box.

In [ ]:
# find the center of the ligand
import MDAnalysis as mda

original_structure = mda.Universe("protein_structures/2zq2.pdb")
ligand_mda = original_structure.select_atoms("resname 13U")

# Get the center of the ligand as the "pocket center"
pocket_center = ligand_mda.center_of_geometry()
print(pocket_center)

After defining the pocket center, define the ligand box.
One simple approach to this is to subtract the min and max of the ligand positions in each dimension.
In order to allow for ligand flexibility and potential interactions with nearby residues, add an additional five angstroms to each side of the box.

In [ ]:
# compute min and max coordinates of the ligand
# take the ligand box to be the difference between the max and min in each direction.
ligand_box = ligand_mda.positions.max(axis=0) - ligand_mda.positions.min(axis=0) + 5
ligand_box

The `pocket_center` and `ligand_box` variables are NumPy arrays.
However, AutoDock Vina expects them to be lists.
Convert them to lists in the cell below.

In [ ]:
pocket_center = pocket_center.tolist()
ligand_box = ligand_box.tolist()

<div class="alert alert-block alert-danger">
<strong>Error in the cell above?</strong>  

If you execute the cell above this one more than once, you will see an error occur.
This happens because on first execution, `pocket_center` and `ligand_box` are NumPy arrays and they have methods to convert to lists.
After you've executed this once, `pocket_center` and `ligand_box` don't have a `tolist` method because they are already lists.

If you execute the cell twice and see an error, you can continue with the rest of the notebook because the variables have already been converted.

</div>

## Docking Ligands with AutoDock Vina

At this stage, PDBQT files of the protein and ligand have been generated and docking box have been defined, ready to perform the actual docking.
Before docking, create a directory to store the results.

In [ ]:
# make a directory to store our results
import os

pdb_id = "2zq2"
ligand = "13U"

os.makedirs("docking_results", exist_ok=True)

Dock using the AutoDock Vina Python API.
First, import `Vina` from `vina`.
Start docking with the line `v = Vina(sf_name="vina")`.
This creates a docking calculation, `v`, and sets the scoring function to the `vina` scoring function.

In [ ]:
from vina import Vina
v = Vina(sf_name="vina")

Then, set the files for the ligand and receptor. Will dock just the ideal ligand first. There are two parameters to docking, the `exhaustiveness` and `n_poses`. The default exhaustiveness value is 8; increasing this to 32 will give a more consistent docking result.

In [ ]:
v.set_receptor(f"pdbqt/{pdb_id}.pdbqt")
v.set_ligand_from_file(f"pdbqt/{ligand}.pdbqt")
v.compute_vina_maps(center=pocket_center, box_size=ligand_box)
v.dock(exhaustiveness=5, n_poses=5)

After the `dock` function, write the poses that were calculated to a file.
Note that the output format from AutoDock Vina is a PDBQT file.

In [ ]:
v.write_poses(f"docking_results/{ligand}.pdbqt", n_poses=5, overwrite=True)

To see the energies of the calculated poses, call `energies` on the docking calculation variable.
According to the Vina documentaiton, the rows correspond to the poses, while columns correspond to different energy types.
The types of energies in the columns are `["total", "inter", "intra", "torsions", "intra best pose"]`.
The number of columns and the types of energies they represent depend on the scoring function used.

In [ ]:
v.energies()

Save these energies to return to them later.
The cell below creates a pandas dataframe and saves the energies as a comma-separated-value (CSV) file.

In [ ]:
import pandas as pd


# These are the columns for the types of energies according to AutoDock Vina docs.
column_names = ["total", "inter", "intra", "torsions", "intra best pose"]

df = pd.DataFrame(v.energies(), columns=column_names)
df.head()

In [ ]:
# Save the calculated energies from docking to a CSV file
df.to_csv("docking_results/13U_energies.csv", index=False)

## Visualizing Docking Results

After performing the docking simulation and saving the energies, visualize the poses.
When visualizing results from molecular docking, scientists often visually inspect the 3D docked structure as well as a 2D representation called an interaction map.
Can use a software called ProLIF (Protein-Ligand Interaction Fingerprints) to make and view these maps in the Jupyter notebook.

To generate these visualizations, convert the files (again!) to the correct format.

In the step above, the poses were written to the file `docking_results/AZO.pdbqt`.
AutoDock Vina only writes in this file, but in order to visualize results, a more standard format is needed.
Use meeko again to convert the poses to an SDF.
Note that meeko will only convert pdbqt files if it prepared the input docking files, which is one reason I used it in the previous notebook.

Again, use a command line script to convert the poses.

In [ ]:
! mk_export.py docking_results/13U.pdbqt -s docking_results/13U.sdf # In the original workshop, `mk_export.py` used the option `-o` instead of `-s`. This was changed to `-s` in late 2024.

After converting to SDF, again visualize the results with ProLIF.
ProLIF requires that molecules be loaded in and has functions to load molecules in several ways.
Use MDAnalysis for loading the proteins to ProLIF and `sdf_supplier` to load the SDFs converted in the previous step.

In [ ]:
import prolif as plf
import MDAnalysis as mda

pdb_id = "2zq2"

protein = mda.Universe(f"protein_structures/protein_h.pdb")

Next, load the protein and ligand into ProLIF.
The function to do this depends on the format of the input data.

In [ ]:
protein_plf = plf.Molecule.from_mda(protein)
poses_plf = plf.sdf_supplier("docking_results/13U.sdf")

To analyze the interactions of the ligand and protein, create a molecular fingerprint object.
By default, ProLIF will calculate nine types of interactions: 'Hydrophobic', 'HBAcceptor', 'HBDonor', 'Cationic', 'Anionic', 'CationPi', 'PiCation', 'PiStacking', 'VdWContact'.

In [ ]:
fp = plf.Fingerprint(count=True)

Next, run ProLif on the poses.
To do this calculation, pass in the list of poses (`poses_plf`) and the ProLIF protein.

In [ ]:
# run on your poses
fp.run_from_iterable(poses_plf, protein_plf)

After running this analysis, visualize the interaction results.
Here is using the 2D and 3D visualization maps, but there are [many other types of analysis](https://prolif.readthedocs.io/en/latest/notebooks/docking.html#analysis) that can be performed.

In [ ]:
pose_index=1

In [ ]:
fp.plot_lignetwork(poses_plf[pose_index])

In [ ]:
view = fp.plot_3d(
    poses_plf[pose_index], protein_plf, frame=pose_index, display_all=False
)
view

<div class="alert alert-block alert-warning">
<h3>Exercise</h3>

Try docking one of the ligands we modified in the previous notebook. Does it bind better or worse according to the docking score? Are the interactions different for the poses?
</div>